# Notatnik analizujący obliczone odległości między szeregami


### Import bibliotek

In [18]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from plotnine import ggplot, aes, geom_density, ggsave, facet_wrap, theme_bw
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
import zipfile

### Pobranie danych z zipa jeżeli folder nie istnieje

In [31]:
path = "../SimilaritiesData/"
if not os.path.exists(path):
  os.makedirs(path)
  with zipfile.ZipFile('../SimilaritiesData.zip', 'r') as zip_ref:
    zip_ref.extractall("../")

### Odczyt odległości z plików

In [20]:
smooth_distance_df = pd.read_csv("../SimilaritiesData/smooth_comparison_distance_all.csv")
normalized_distance_df = pd.read_csv("../SimilaritiesData/normalized_comparison_distance_all.csv")
distance_df = pd.read_csv("../SimilaritiesData/comparison_distance_all.csv")
normalized_smooth_distance_df = pd.read_csv("../SimilaritiesData/normalized_smooth_comparison_distance_all.csv")

smooth_distance_df.reset_index(drop=True, inplace=True)
normalized_distance_df.reset_index(drop=True, inplace=True)
distance_df.reset_index(drop=True, inplace=True)
normalized_smooth_distance_df.reset_index(drop=True, inplace=True)

In [21]:
def clean(distance_data):
  distance_data['same_functions'] = distance_data['function1'] == distance_data['function2']
  distance_data['same_udf'] = distance_data['udf1'] == distance_data['udf2']
  distance_data['lcss_distance'] = 1 - distance_data['lcss_similarity']
  distance_data.drop(distance_data[distance_data.snapshot1 == distance_data.snapshot2].index, inplace=True)
  distance_data["function1"].replace({"filtration": "filtering",
                                    "filtration-aggregation": "filtering-aggregation",
                                     "filtration-aggregation-join": "filtering-aggregation-join",
                                    "filtration-join": "filtering-join"}, inplace=True)
  distance_data["function2"].replace({"filtration": "filtering",
                                    "filtration-aggregation": "filtering-aggregation",
                                    "filtration-aggregation-join": "filtering-aggregation-join",
                                    "filtration-join": "filtering-join"}, inplace=True)

  return distance_data

In [22]:
distance_df = clean(distance_df)
normalized_distance_df = clean(normalized_distance_df)
smooth_distance_df = clean(smooth_distance_df)
normalized_smooth_distance_df = clean(normalized_smooth_distance_df)

 ### Utworzenie filtrów

In [23]:
filterCPU = distance_df["dataType"]=="CPU"
filterRAM = distance_df["dataType"]=="RAM"
filterDifferent = distance_df["function1"]!=distance_df["function2"]
filterSame = distance_df["function1"]==distance_df["function2"]
filterDifferentSize = distance_df["size1"]!=distance_df["size2"]
filterSameSize = distance_df["size1"]==distance_df["size2"]
filterSize1 = distance_df["size1"]==1
filterSize2 = distance_df["size2"]==2
filterDTWValueMoreThan1000 = distance_df["dtw-python-distance"] > 1000

filterAggregation1 = distance_df["function1"]=="aggregation"
filterAggregation2 = distance_df["function2"]=="aggregation"
filterFiltration1 = distance_df["function1"]=="filtration"
filterFiltration2 = distance_df["function2"]=="filtration"
filterFiltrationAggregation1 = distance_df["function1"]=="filtration-aggregation"
filterFiltrationAggregation2 = distance_df["function2"]=="filtration-aggregation"
filterFiltrationAggregationJoin1 = distance_df["function1"]=="filtration-aggregation-join"
filterFiltrationAggregationJoin2 = distance_df["function2"]=="filtration-aggregation-join"
filterFiltrationJoin1 = distance_df["function1"]=="filtration-join"
filterFiltrationJoin2 = distance_df["function2"]=="filtration-join"

Utworzenie folderu na wyniki

In [28]:
pathSimilarities = "../Plots/similarities/plots"
pathHeatmaps = "../Plots/similarities/heatmaps"
if not os.path.exists(pathSimilarities):
  os.makedirs(pathSimilarities)
if not os.path.exists(pathHeatmaps):
  os.makedirs(pathHeatmaps)

### Funkcja tworząca wykresy gęstości

In [29]:
from plotnine.themes.themeable import legend_position
from plotnine import theme, element_line


def make_plot(function, distance_data, data_state):
  function_file_name = ''
  if function == 'euclidean': function_file_name = 'euclidean'
  if function == 'dtw-python-distance': function_file_name = 'dtw'
  if function == 'lcss_distance': function_file_name = 'lcss'
  cpu_plot = (ggplot(distance_data[filterCPU], aes(x=function, color='function2', fill='function2'))
  + geom_density(alpha=0.1)
  + facet_wrap('function1', ncol=1)
    + theme_bw()
  + theme(legend_position =(.5, -0.06), legend_direction='horizontal', legend_title_align="center", panel_grid=element_line(color="lightgrey"))
)
  ram_plot = (ggplot(distance_data[filterRAM], aes(x=function, color='function2', fill='function2'))
  + geom_density(alpha=0.1)
  + facet_wrap('function1', ncol=1)
  + theme_bw()
  + theme(legend_position =(.5, -0.06), legend_direction='horizontal', legend_title_align="center", panel_grid=element_line(color="lightgrey"))
  )

  ggsave(plot=cpu_plot, filename=f"../Plots/similarities/plots/cpu_"+function_file_name+"_"+data_state+".pdf", dpi=1000)
  ggsave(plot=ram_plot, filename=f"../Plots/similarities/plots/ram_"+function_file_name+"_"+data_state+".pdf", dpi=1000)

### Wykresy gęstości - odległość euklidesowa

In [30]:
make_plot('euclidean', distance_df, 'default')
make_plot('euclidean', smooth_distance_df, 'smooth')
make_plot('euclidean', normalized_distance_df, 'normalized')
make_plot('euclidean', normalized_smooth_distance_df, 'normalized_smooth')

KeyboardInterrupt: 

### Wykresy gęstości - odległość DTW


In [ ]:
make_plot('dtw-python-distance', distance_df, 'default')
make_plot('dtw-python-distance', smooth_distance_df, 'smooth')
make_plot('dtw-python-distance', normalized_distance_df, 'normalized')
make_plot('dtw-python-distance', normalized_smooth_distance_df, 'normalized_smooth')

### Wykresy gęstości - odległość LCSS


In [ ]:
make_plot('lcss_distance', distance_df, 'default')
make_plot('lcss_distance', smooth_distance_df, 'smooth')

### Heatmapy - odległość euklidesowa

In [ ]:
#Default
sns.heatmap(pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU].euclidean, 
                 aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_euclidean_cpu_default.pdf', bbox_inches='tight', dpi=300)
plt.clf()

sns.heatmap(pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM].euclidean, 
                 aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')
plt.savefig('../SimilaritiesData/heatmaps/heatmap_euclidean_ram_default.pdf', bbox_inches='tight', dpi=300)
plt.clf()

#Smooth
sns.heatmap(pd.crosstab(smooth_distance_df.function1, smooth_distance_df.function2,
                 values=smooth_distance_df[filterCPU].euclidean, aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_euclidean_cpu_smooth.pdf', bbox_inches='tight', dpi=300)
plt.clf()

sns.heatmap(pd.crosstab(smooth_distance_df.function1, smooth_distance_df.function2,
                 values=smooth_distance_df[filterRAM].euclidean, aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_euclidean_ram_smooth.pdf', bbox_inches='tight', dpi=300)
plt.clf()

#Normalized
sns.heatmap(pd.crosstab(normalized_distance_df.function1, normalized_distance_df.function2,
                 values=normalized_distance_df[filterCPU].euclidean, aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_euclidean_cpu_normalized.pdf', bbox_inches='tight', dpi=300)
plt.clf()

sns.heatmap(pd.crosstab(normalized_distance_df.function1, normalized_distance_df.function2,
                 values=normalized_distance_df[filterRAM].euclidean, aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_euclidean_ram_normalized.pdf', bbox_inches='tight', dpi=300)
plt.clf()

#Normalized-smooth
sns.heatmap(pd.crosstab(normalized_smooth_distance_df.function1, normalized_smooth_distance_df.function2,
                 values=normalized_smooth_distance_df[filterCPU].euclidean, aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_euclidean_cpu_normalized_smooth.pdf', bbox_inches='tight', dpi=300)
plt.clf()

sns.heatmap(pd.crosstab(normalized_smooth_distance_df.function1, normalized_smooth_distance_df.function2,
                 values=normalized_smooth_distance_df[filterRAM].euclidean, aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_euclidean_ram_normalized_smooth.pdf', bbox_inches='tight', dpi=300)
plt.clf()


### Heatmapy - odległość DTW


In [ ]:
#Default
sns.heatmap(pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU]['dtw-python-distance'], 
                 aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_dtw_cpu_default.pdf', bbox_inches='tight', dpi=300)
plt.clf()

sns.heatmap(pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM]['dtw-python-distance'], 
                 aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')
plt.savefig('../SimilaritiesData/heatmaps/heatmap_dtw_ram_default.pdf', bbox_inches='tight', dpi=300)
plt.clf()

#Smooth
sns.heatmap(pd.crosstab(smooth_distance_df.function1, smooth_distance_df.function2,
                 values=smooth_distance_df[filterCPU]['dtw-python-distance'], aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_dtw_cpu_smooth.pdf', bbox_inches='tight', dpi=300)
plt.clf()

sns.heatmap(pd.crosstab(smooth_distance_df.function1, smooth_distance_df.function2,
                 values=smooth_distance_df[filterRAM]['dtw-python-distance'], aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_dtw_ram_smooth.pdf', bbox_inches='tight', dpi=300)
plt.clf()

#Normalized
sns.heatmap(pd.crosstab(normalized_distance_df.function1, normalized_distance_df.function2,
                 values=normalized_distance_df[filterCPU]['dtw-python-distance'], aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_dtw_cpu_normalized.pdf', bbox_inches='tight', dpi=300)
plt.clf()

sns.heatmap(pd.crosstab(normalized_distance_df.function1, normalized_distance_df.function2,
                 values=normalized_distance_df[filterRAM]['dtw-python-distance'], aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_dtw_ram_normalized.pdf', bbox_inches='tight', dpi=300)
plt.clf()

#Normalized-smooth
sns.heatmap(pd.crosstab(normalized_smooth_distance_df.function1, normalized_smooth_distance_df.function2,
                 values=normalized_smooth_distance_df[filterCPU]['dtw-python-distance'], aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_dtw_cpu_normalized_smooth.pdf', bbox_inches='tight', dpi=300)
plt.clf()

sns.heatmap(pd.crosstab(normalized_smooth_distance_df.function1, normalized_smooth_distance_df.function2,
                 values=normalized_smooth_distance_df[filterRAM]['dtw-python-distance'], aggfunc='mean').round(2), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_dtw_ram_normalized_smooth.pdf', bbox_inches='tight', dpi=300)
plt.clf()


### Heatmapy - odległość LCSS


In [ ]:
#Default
sns.heatmap(pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU]['lcss_distance'], 
                 aggfunc='mean').round(4), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_lcss_cpu_default.pdf', bbox_inches='tight', dpi=300)
plt.clf()

sns.heatmap(pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM]['lcss_distance'], 
                 aggfunc='mean').round(4), 
            cmap="flare", annot=True,  fmt='g')
plt.savefig('../SimilaritiesData/heatmaps/heatmap_lcss_ram_default.pdf', bbox_inches='tight', dpi=300)
plt.clf()

#Smooth
sns.heatmap(pd.crosstab(smooth_distance_df.function1, smooth_distance_df.function2,
                 values=smooth_distance_df[filterCPU]['lcss_distance'], aggfunc='mean').round(4), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_lcss_cpu_smooth.pdf', bbox_inches='tight', dpi=300)
plt.clf()

sns.heatmap(pd.crosstab(smooth_distance_df.function1, smooth_distance_df.function2,
                 values=smooth_distance_df[filterRAM]['lcss_distance'], aggfunc='mean').round(4), 
            cmap="flare", annot=True,  fmt='g')

plt.savefig('../SimilaritiesData/heatmaps/heatmap_lcss_ram_smooth.pdf', bbox_inches='tight', dpi=300)
plt.clf()

### Analiza odległości dla różnych rozmiarów - odległość euklidesowa CPU

In [ ]:
size1 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterSameSize & filterSize1]['euclidean'], 
                 aggfunc='mean')
size2 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterSameSize & filterSize2]['euclidean'], 
                 aggfunc='mean')
differentSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterDifferentSize]['euclidean'], 
                 aggfunc='mean')
sameSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterSameSize]['euclidean'], 
                 aggfunc='mean')

size1size2 = size1.subtract(size2).div(size1).mul(100)
size1size2 = size1size2.where(size1size2.isna(), size1size2.round(2).astype(str).add('%'))
differentSizeSameSize = differentSize.subtract(sameSize).div(differentSize).mul(100)
differentSizeSameSize = differentSizeSameSize.where(differentSizeSameSize.isna(), differentSizeSameSize.round(2).astype(str).add('%'))
print(size1size2.to_latex())  
print(differentSizeSameSize.to_latex())

### Analiza odległości dla różnych rozmiarów - odległość euklidesowa RAM


In [ ]:
size1 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterSameSize & filterSize1]['euclidean'],
                 aggfunc='mean')
size2 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterSameSize & filterSize2]['euclidean'],
                 aggfunc='mean')
differentSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterDifferentSize]['euclidean'],
                 aggfunc='mean')
sameSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterSameSize]['euclidean'],
                 aggfunc='mean')

size1size2 = size1.subtract(size2).div(size1).mul(100)
size1size2 = size1size2.where(size1size2.isna(), size1size2.round(2).astype(str).add('%'))
differentSizeSameSize = differentSize.subtract(sameSize).div(differentSize).mul(100)
differentSizeSameSize = differentSizeSameSize.where(differentSizeSameSize.isna(), differentSizeSameSize.round(2).astype(str).add('%'))
print(size1size2.to_latex())
print(differentSizeSameSize.to_latex())

### Analiza odległości dla różnych rozmiarów - odległość DTW CPU


In [ ]:
size1 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterSameSize & filterSize1]['dtw-python-distance'], 
                 aggfunc='mean')
size2 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterSameSize & filterSize2]['dtw-python-distance'], 
                 aggfunc='mean')
differentSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterDifferentSize]['dtw-python-distance'], 
                 aggfunc='mean')
sameSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterSameSize]['dtw-python-distance'], 
                 aggfunc='mean')

size1size2 = size1.subtract(size2).div(size1).mul(100)
size1size2 = size1size2.where(size1size2.isna(), size1size2.round(2).astype(str).add('%'))
differentSizeSameSize = differentSize.subtract(sameSize).div(differentSize).mul(100)
differentSizeSameSize = differentSizeSameSize.where(differentSizeSameSize.isna(), differentSizeSameSize.round(2).astype(str).add('%'))
print(size1size2.to_latex())  
print(differentSizeSameSize.to_latex())

### Analiza odległości dla różnych rozmiarów - odległość DTW RAM


In [ ]:
size1 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterSameSize & filterSize1]['dtw-python-distance'], 
                 aggfunc='mean')
size2 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterSameSize & filterSize2]['dtw-python-distance'], 
                 aggfunc='mean')
differentSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterDifferentSize]['dtw-python-distance'], 
                 aggfunc='mean')
sameSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterSameSize]['dtw-python-distance'], 
                 aggfunc='mean')

size1size2 = size1.subtract(size2).div(size1).mul(100)
size1size2 = size1size2.where(size1size2.isna(), size1size2.round(2).astype(str).add('%'))
differentSizeSameSize = differentSize.subtract(sameSize).div(differentSize).mul(100)
differentSizeSameSize = differentSizeSameSize.where(differentSizeSameSize.isna(), differentSizeSameSize.round(2).astype(str).add('%'))
print(size1size2.to_latex())
print(differentSizeSameSize.to_latex())

### Analiza odległości dla różnych rozmiarów - odległość LCSS CPU


In [ ]:
size1 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterSameSize & filterSize1]['lcss_distance'], 
                 aggfunc='mean')
size2 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterSameSize & filterSize2]['lcss_distance'], 
                 aggfunc='mean')
differentSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterDifferentSize]['lcss_distance'], 
                 aggfunc='mean')
sameSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterCPU & filterSameSize]['lcss_distance'], 
                 aggfunc='mean')

size1size2 = size1.subtract(size2).div(size1).mul(100)
size1size2 = size1size2.where(size1size2.isna(), size1size2.round(2).astype(str).add('%'))
differentSizeSameSize = differentSize.subtract(sameSize).div(differentSize).mul(100)
differentSizeSameSize = differentSizeSameSize.where(differentSizeSameSize.isna(), differentSizeSameSize.round(2).astype(str).add('%'))
print(size1size2.to_latex())  
print(differentSizeSameSize.to_latex())

### Analiza odległości dla różnych rozmiarów - odległość LCSS RAM


In [ ]:
size1 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterSameSize & filterSize1]['lcss_distance'], 
                 aggfunc='mean')
size2 = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterSameSize & filterSize2]['lcss_distance'], 
                 aggfunc='mean')
differentSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterDifferentSize]['lcss_distance'], 
                 aggfunc='mean')
sameSize = pd.crosstab(distance_df.function1, distance_df.function2,
                 values=distance_df[filterRAM & filterSameSize]['lcss_distance'], 
                 aggfunc='mean')

size1size2 = size1.subtract(size2).div(size1).mul(100)
size1size2 = size1size2.where(size1size2.isna(), size1size2.round(2).astype(str).add('%'))
differentSizeSameSize = differentSize.subtract(sameSize).div(differentSize).mul(100)
differentSizeSameSize = differentSizeSameSize.where(differentSizeSameSize.isna(), differentSizeSameSize.round(2).astype(str).add('%'))
print(size1size2.to_latex())  
print(differentSizeSameSize.to_latex())

### Wykresy gęstości dla DTW - bez wartości odstających

In [ ]:
filterDTWValueLessThan5000 = smooth_distance_df["dtw-python-distance"] < 6000

plot = (ggplot(smooth_distance_df[filterDTWValueLessThan5000 & filterCPU], aes(x='dtw-python-distance', color='function2', fill='function2'))
  + geom_density(alpha=0.1)
  + facet_wrap('function1', ncol=1)
    + theme_bw()
  + theme(legend_position =(.5, -0.06), legend_direction='horizontal', legend_title_align="center", panel_grid=element_line(color="lightgrey"))
)
ggsave(plot=plot, filename=f"../Plots/similarities/plots/cpu_dtw_smooth_less_than_5000.pdf", dpi=1000)

In [ ]:
filterDTWValueLessThan1500 = distance_df["dtw-python-distance"] < 1500

plot = (ggplot(distance_df[filterDTWValueLessThan1500 & filterRAM], aes(x='dtw-python-distance', color='function2', fill='function2'))
  + geom_density(alpha=0.1)
  + facet_wrap('function1', ncol=1)
    + theme_bw()
  + theme(legend_position =(.5, -0.06), legend_direction='horizontal', legend_title_align="center", panel_grid=element_line(color="lightgrey"))
)
ggsave(plot=plot, filename=f"../Plots/similarities/plots/ram_dtw_less_than_1500.pdf", dpi=1000)

In [ ]:

filterDTWValueLessThan10000 = distance_df["dtw-python-distance"] < 10000


plot = (ggplot(distance_df[filterDTWValueLessThan10000 & filterCPU], aes(x='dtw-python-distance', color='function2', fill='function2'))
   + geom_density(alpha=0.1)
    + facet_wrap('function1', ncol=1)
    + theme_bw()
  + theme(legend_position =(.5, -0.06), legend_direction='horizontal', legend_title_align="center", panel_grid=element_line(color="lightgrey"))
)
ggsave(plot=plot, filename=f"../Plots/similarities/plots/cpu_dtw_less_than_10000.pdf", dpi=1000)